In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()     # versions that might influence the numerical results, matplotlib and seaborn are excluded 
sc.settings.set_figure_params(dpi=80, facecolor='white') 

## analysis

In [ ]:
data = sc.read_10x_mtx('raw_data')

In [ ]:
data.var_names_make_unique() 

In [ ]:
sc.pl.highest_expr_genes(data, n_top=20) #shows the top 20 expressed genes

In [ ]:
sc.pp.filter_cells(data, min_genes=300) #basic filtering, filtering out the cells with less than 300 genes and genes found 
sc.pp.filter_genes(data, min_cells=3)                        # less than 3 cells

In [ ]:
data.var['mt'] = data.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(data, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True) 
    #Set to False to skip computing log1p transformed annotations
sc.pl.violin(data, 'n_genes_by_counts')
#plots - numbers of expressed genes - bt 400 and 2500? 3000?
#2 ->total counts = cell size = counts of genes for each cells
#3 -> percent of reads mapped to genes in the mitochondrial genome



In [ ]:
sc.pl.violin(data, 'total_counts') #jitter=0.4


sc.pl.violin(data, 'pct_counts_mt')

In [ ]:
sc.pl.scatter(data, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(data, x='total_counts', y='n_genes_by_counts')

In [ ]:
data = data[data.obs.n_genes_by_counts < 3000, :] #removing cells with too many counts 
#looking at the first plot, i chose 10 v and 3000 ^ but maybe it should actually be bit lower?
data = data[data.obs.pct_counts_mt < 10, :]  #removing cells with too much mitochondrial genes

In [ ]:
sc.pp.normalize_total(data, target_sum=1e4) #normalization, target_sum=1e4 -> CPM normalization

In [ ]:
sc.pp.log1p(data) 

In [ ]:
sc.pp.highly_variable_genes(data, min_mean=0.0125, max_mean=3, min_disp=0.5) #identify highly-variable genes, 
                                                                                #expect log data

In [ ]:
sc.pl.highly_variable_genes(data)

In [ ]:
data.raw = data

In [ ]:
data.obs.n_genes

In [ ]:
data = data[:, data.var.highly_variable]

In [ ]:
sc.pp.regress_out(data, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(data, max_value=10)

In [ ]:
#PCA -> reduce the dimensiality of data -> smaller data sets are easier to explore and visualize
sc.tl.pca(data, n_comps=50, svd_solver='arpack')
#sc.pl.pca(data, color='MALAT1')
sc.pl.pca_overview(data, components=['1,2', '2,3'])

In [ ]:
sc.pl.pca_overview(data, color=["CD68", "LYZ", "GZMB"])

In [ ]:
sc.pl.pca_variance_ratio(data, log=True,  n_pcs=50) #to know how many PCs should be considered to compute the neighbourhood
                                            #relations of cells

In [ ]:
sc.pp.neighbors(data, n_neighbors=10, n_pcs=8) #looking at the plot above i decided to choose 22 PC as the plot is 
sc.tl.umap(data)                                   #flattening there

In [ ]:
sc.tl.leiden(data, key_added='leiden_res0_5', resolution=0.5)
sc.pl.umap(data, color='leiden_res0_5', legend_fontsize=8)

In [ ]:
#finding the marker genes 
            
sc.tl.rank_genes_groups(data, 'leiden_res0_5', method='t-test') #by default a .raw data is used
sc.pl.rank_genes_groups(data, n_genes=25, sharey=False)
#that is a ranking for the highly differential genes in each cluster

## annotating

In [ ]:
# my annotation
# markers from Human resident liver myeloid cells protect against metabolic stress in obesity
marker_genes_grouped = {'myeloid cells' : ['CD68', 'LYZ', 'CD14', 'LGALS3'],
                'cDC1' : ['XCR1', 'CLEC9A', 'FLT3'],
                'mast cells' : ['KIT', 'ENPP3'],
                'B cells' : ['CD19', 'MS4A1', 'CD79A'],
                'T cells' : ['CD3G', 'CD3D'],
                'resident NK cells' : ['NCR1', 'KLRF1', 'CD7'],
                'circulating NK and NKT cells' : ['GZMB', 'GNLY', 'FCGR3A'],
                'endothelial cells': ['ENG', 'MCAM', 'LYVE1'],
                'proliferating cells' : ['CDK1', 'CCNA2', 'PCNA'],
                       'cholangiocytes' : ['EPCAM', 'SPP1', 'DDIT4L', 'SLC5A1', 'KRT7', 'KRT19', 'SOX9'],
                       'stromal cells' : ['CARMN'],
                       'plasma cells' : ['SDC1', 'MZB1', 'IGHG1'], #IGKC and IGLC2 slabe bo wszedzie widac
                       'hepatocytes' : ['APOA2', 'TTR', 'CYP2E1', 'HAL', 'SDS'],
                        'endothelial cells': ['ENG', 'MCAM', 'LYVE1', 'PECAM1','OIT3', 'F8','C1QTNF1', 'MMRN2', 'PCDH12'],
                     'smooth muscle cells' : ['ACTA2'],
                     'fibroplasts' : ['COL1A1', 'PDGFRA', 'COL1A2', 'CD34', 'FBLN1', 'COL5A1', 'LOXL1', 'LUM', 'FBLN1', 'FBLN2']
                       }
                      

marker_genes = ['CD68', 'LYZ', 'CD14', 'LGALS3', 'XCR1', 'CLEC9A', 'FLT3', 'KIT', 'ENPP3', 'CD19', 'MS4A1', 'CD79A',
               'CD3G', 'CD3D', 'NCR1', 'KLRF1', 'CD7', 'GZMB', 'GNLY', 'FCGR3A', 'ENG', 'MCAM', 'LYVE1', 'CDK1', 'CCNA2', 'PCNA']

In [ ]:
sc.pl.dotplot(data, marker_genes_grouped, groupby='leiden_res0_5')

In [ ]:
sc.pl.dotplot(data, marker_genes_grouped, groupby='my_annotation')

In [ ]:
#data.write("results_XX.h5ad") 

In [ ]:
sc.pl.dotplot(data, {'endothelial cells': ['ENG', 'MCAM', 'LYVE1', 'PECAM1','OIT3', 'F8','C1QTNF1', 'MMRN2', 'PCDH12'],
                     'smc' : ['ACTA2'],
                     'fibroplasts' : ['COL1A1', 'PDGFRA', 'COL1A2', 'CD34', 'FBLN1', 'COL5A1', 'LOXL1', 'LUM', 'FBLN1', 'FBLN2']},
                     groupby='my_annotation')

In [ ]:
sc.pl.dotplot(data, marker_genes_grouped, groupby='my_annotation')

In [ ]:
marker_heatmap_NK = {'B cells' : ['CD19', 'MS4A1', 'CD79A'],
                    'NK cells' : ['GZMB', 'GNLY', 'KLRF1', 'CD7'],
                    'T cells' : ['CD3G', 'CD3D'],
                    'cholangiocytes' : ['EPCAM', 'SPP1', 'KRT7', 'KRT19'],
                    'endothelial cells': ['ENG', 'MCAM', 'LYVE1'],
                    'fibroblasts' : ['COL1A1', 'COL1A2', 'FBLN1'],
                    'myeloid cells' : ['CD68', 'LYZ', 'CD14', 'LGALS3'],
                    'pDC' : [ 'JCHAIN' , 'GZMB'], 
                    'plasma cells' : ['SDC1', 'MZB1', 'IGHG1'],#IGKC and IGLC2 slabe bo wszedzie widac
                    'smooth muscle cells' : ['ACTA2', 'ADIRF']
                }

In [ ]:
sc.pl.heatmap(data, marker_heatmap_NK, groupby='my_annotation', show_gene_labels=True,  log=True, cmap='Reds')

In [ ]:
sc.pl.matrixplot(data, marker_heatmap_NK, groupby='my_annotation', vmin=-2, vmax=2, cmap='RdBu')

In [ ]:
marker_genes_grouped = {'myeloid cells' : ['CD68', 'LYZ',  'LGALS3'], #'CD14',
                   'cDC1' : ['XCR1'], #'CLEC9A', ,  'FLT3'
                   'mast cells' : ['KIT'], #, 'ENPP3'
                  'B cells' : [ 'MS4A1', 'CD79A'], #'CD19',
                'T cells' : ['CD3G', 'CD3D'],
                'resident NK cells' : ['KLRF1', 'CD7'], #'NCR1', 
                'circulating NK and NKT cells' : ['GZMB', 'GNLY', 'FCGR3A'],
                'proliferating cells' : [ 'PCNA'], #'CCNA2' 'CDK1', 
                   'cholangiocytes' : [ 'SPP1',   'KRT7', 'KRT19'], #'DDIT4L', 'EPCAM', 'SLC5A1', , 'SOX9'
                    'stromal cells' : ['CARMN'],
                   'plasma cells' : [ 'MZB1', 'IGHG1'], #IGKC and IGLC2 slabe bo wszedzie widac #SDC1',
                   'hepatocytes' : ['APOA2', 'TTR', 'CYP2E1' ], #'HAL','SDS'
                    'endothelial cells': ['ENG', 'LYVE1','OIT3', 'F8'], #'C1QTNF1', 'MMRN2', , 'PCDH12  'PECAM1', 'MCAM', 
                     'smooth muscle cells' : [ 'ACTA2'], #'PALLD',
                     'fibroplasts' : ['COL1A1',  'COL1A2', 'CD34',  'LUM', 'FBLN1', ], #'COL5A1', 'FBLN2' 'LOXL1','PDGFRA',
                    'erythroid cells' : ['HBB', 'CA1'] 
                       }
                      
sc.pl.dotplot(data, marker_genes_grouped, groupby='leiden_res0_7')

In [ ]:
annotation = {
     '0': '',                  
     '1': '',                           
     '2': '',
     '3': '',               
     '4': '',      
    '5' : '',                     
    '6' : '',                 
    '7' : '',                     
    '8' : '',                          
    '9' : '',                     
    '10' : '',     
    '11' : '',                    
    '12' : '',
    '13' : '',
    '14' : '',
    '15' : ''}             

data.obs['my_annotation'] = data.obs['leiden_res0_5'].map(annotation1).astype('category')

sc.pl.umap(data, color=['my_annotation'] )  